#   ROS2 | Exercise 4 - Computer Vision

##  Task 3 - Object Detection

### Pre-requisites
Before you begin this Notebook you have to complete the following tasks explained in the handout
- [x] Setting up the gazebo simulation as a ROS package
- [x] Setting up Conda environment to run this jupyter notebook

### Introduction
Object detection and its importance in robotics. Opencv Intro. Some images

There are 3 parts in this Notebook which covers the following
   - 3.1 : Basic Image processing and Opencv contour detection
   - 3.2 : Object detection and Tracking with Video
   - 3.3 : Object detection and Tracking with ROS2

***
### 3.1 Basic Image processing with OpenCV 

### 3.1.1
Familiarize yourself with the basic functionalities available in openCV for basic image reading and visualization with this reference [link](https://docs.opencv.org/4.x/db/deb/tutorial_display_image.html). It has both C++ & Python code. We are following python in this notebook. Execute the following code that reads an image file and displays it in an opencv window.

In [1]:
# Libraries
import sys 
import imutils
import cv2
import numpy as np

# Variables
windowName1 = 'Tracking'
windowName2 = 'HSV Frame'

windowSizeX = 950
windowSizeY = 700

# Create default window and resize it
# Tracking frame
cv2.namedWindow(windowName1,cv2.WINDOW_NORMAL)
cv2.resizeWindow(windowName1, windowSizeX,windowSizeY)

# HSV frame
cv2.namedWindow(windowName2,cv2.WINDOW_NORMAL)
cv2.resizeWindow(windowName2, windowSizeX,windowSizeY)

# Image reading function. NOTICE! By default, the image is read in BGR,NOT in RGB
frame = cv2.imread('image.png')

# Image processing
# Color Space Conversions
hsvFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)


# Create window(s) whith the title and image
cv2.imshow(windowName1,frame)
cv2.imshow(windowName2,hsvFrame)

# Wait until user presses any key -> destroy window(s)
cv2.waitKey(0) & 0xFF
cv2.destroyAllWindows()

# 3.1.2

It is important to  pre-process raw input images in computer vision tasks such as object detection and tracking. OpenCV imgproc module contains several functionalities which are often used in image processing.

Convertion between color spaces is necessary to generate image masks. Execute and observe the following code where you convert an image from RGB to HSV color space. The code generates a mask of the green color object.

**Your task**

- [x] Change HSV values in order to generate masks of other objects.

![RGB vs. HSV](RGB-left-and-HSV-right-color-spaces.png) 

Acronyms and short descriptions:
- HSL (for hue, saturation, lightness) and
- HSV (for hue, saturation, value; also known as  HSB (V=B)
- HSB, for hue, saturation, brightness) 

HSL/HSV are alternative representations of the RGB color 

*Hue* is a colour or shade. Hue is usually a number between 0 and 360 
which represents the color wheel.

*Lightness* is the amount of white color introduced to the color [0-1]

*Saturation* pertains the amount of white light mixed with a hue [0-1]

*Brightness and value* represents the perception of the 
ammount of light or power of the source. [0-1]

**Why converting images from BGR -> HSV ?** 
"*The simple answer is that unlike RGB, HSV separates luma, or the image intensity, from chroma or the color information. This is very useful in many applications, and is used as masking. For example, if you want to do histogram equalization of a color image, you probably want to do that only on the intensity component, and leave the color components alone. Otherwise you will get very strange colors. In computer vision you often want to separate color components from intensity for various reasons, such as robustness to lighting changes, or removing shadows.*"  - StackExchange, user Dima

**Nice Video by Ben (Learn Code By Gaming), Where the HSV and thresholing is explained well with graphical material** https://www.youtube.com/watch?v=0tKzqsRtmyY [13 min -->]

**References**
- [Concept of HSL and HSV Color spaces](https://en.wikipedia.org/wiki/HSL_and_HSV)
- [Opencv basic Image processing functionalities](https://docs.opencv.org/4.x/d7/da8/tutorial_table_of_content_imgproc.html)
- [OpenCV RGB-HSV color converstions documentation](https://docs.opencv.org/3.4.6/de/d25/imgproc_color_conversions.html#color_convert_rgb_hsv)


In [2]:
# lower and upper boundaries for HSV thresholds
# So this means what is the color range which be filtered for the mask

# Green plate + yellow gear (DEFAULT)
Lower = (29, 86, 6)       
Upper = (64, 255, 255)  

# Green plate
# Lower = (40, 86, 6)       
# Upper = (64, 255, 255)    

# Turquoise gear
# Lower = (68, 86, 6)       
# Upper = (90, 255, 255)  

# Red gear
# Lower = (0, 86, 6)       
# Upper = (25, 255, 255) 

# ALL
# Lower = (0, 30, 0)       
# Upper = (255, 255, 255) 


# Preparation function, witch returns original frame and mask
def prep(frame):
    # Resize the frame, with image utilization tool
    frame = imutils.resize(frame, width=600)
    
    ## Todo 3.1.3 gaussian blur
    kernelSizeGaus = 11
    blurred = cv2.GaussianBlur(frame,(kernelSizeGaus,kernelSizeGaus),0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv,Lower,Upper)

    ## Todo 3.1.3 Erosion / Dialation
    kernelSizeEroDia = 3
    eroDiaIterations = 1
    kernel = np.ones((kernelSizeEroDia, kernelSizeEroDia), np.uint8)
    frame_erosion = cv2.erode(frame, kernel, iterations=eroDiaIterations)
    frame_dilation = cv2.dilate(frame, kernel, iterations=eroDiaIterations)
    
    blurEro = cv2.erode(hsv, kernel, iterations=eroDiaIterations)
    blurEroDil = cv2.dilate(blurEro, kernel, iterations=eroDiaIterations)
    blurEroDilMask = cv2.inRange(blurEroDil,Lower,Upper)
    
    return mask, frame, frame_erosion, frame_dilation, blurred, blurEroDil, blurEroDilMask

mask, frame, frame_erosion, frame_dilation, blurred, blurEroDil, blurEroDilMask = prep(frame)

cv2.imshow("Image",frame)
cv2.imshow("Blurred",blurred)
cv2.imshow("Masked Image",mask)
cv2.imshow("Erosion",frame_erosion)
cv2.imshow("Dilation",frame_dilation)
cv2.imshow("Blurred>Erode>Dilate",blurEroDil)
cv2.imshow("Blurred>Erode>Dilate>Mask",blurEroDilMask)
cv2.waitKey(0) & 0xFF
cv2.destroyAllWindows()

### 3.1.3

The above implementation provides a satisfactory result. However, when dealing with noisy images it is important to perform morphological transformations such as smoothing, erotion and dialation to get better results.

**Your task**

Read the following documentations and implement the following transformations inside the prep_frame() function above.


- [ ] Apply Gaussian blur with a kernel size of 11x11
- [ ] Apply erosion
- [ ] Apply dialation

**References**

- [Smoothing Images with OpenCV](https://docs.opencv.org/4.5.2/d4/d13/tutorial_py_filtering.html)
- [Perform Erosion and Dialation on the image mask using OpenCV](https://docs.opencv.org/3.4/db/df6/tutorial_erosion_dilatation.html)

### 3.1.4

Finding contours in an image is useful in object detection and tracking. The code snippet below extracts contours from the image file and draws a circle with its center as the same center of detected contour. Execute the code below and observe the results.

**References**
- [ Detecting contours with OpenCV](https://docs.opencv.org/4.5.0/d4/d73/tutorial_py_contours_begin.html)

In [3]:
### Find contours and display
def find_cnts(mask):
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    return cnts

# Find contours / cnts is list
cnts = find_cnts(mask)

# Search contour with largest area / c in numpy array
c = max(cnts, key=cv2.contourArea)

# Chose what to draw
circle = True
rectangle = True
rectangleRot = True
convexHull = True

# Frame reference
frame1 = frame.copy()

if(circle):
    # It is a circle which completely covers the object with minimum area. 
    ((x, y), radius) = cv2.minEnclosingCircle(c) ## A different contour?

    # Find center of contour using moments in opencvq
    M = cv2.moments(c)
    center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

    # Draw circle
    cv2.circle(frame1, (int(x), int(y)), int(radius),(0, 0, 255), 0)
    cv2.circle(frame1, center, 5, (0, 0, 255), -1)    
    cv2.imshow("Circle", frame1)

## ToDo 3.1.5 
#  1. Bounding rectangle(s)
#  2. Convex Hull

if(rectangle):
    # Calculate xy position and add w and h to them (other corners)
    x,y,w,h = cv2.boundingRect(c)
    
    # Draw the rectangle 
    cv2.rectangle(frame1,(x,y),(x+w,y+h),(255,255,0),1)
    
    cv2.imshow("Rectangle", frame1)

if(rectangleRot):
    # Finds a rotated rectangle of the minimum area enclosing the input 2D point set. 
    rect = cv2.minAreaRect(c)
    
    # Finds the four vertices of a rotated rect. Useful to draw the rotated rectangle. 
    box = cv2.boxPoints(rect)
    
    # Float -> Integer
    box = np.int0(box)
    
    # Draw rectange args(image,contours,contour index, color BGR, thickness, line type ,...)
    cv2.drawContours(frame1,[box],0,(0,255,0),1)
    cv2.imshow("Rectangle Rotate", frame1)
    
if(convexHull):
    # Find the convex hull of a point set
    conhull = cv2.convexHull(c)
    
    # Draw points
    cv2.drawContours(frame1,[conhull],0,(255,0,0),1)
    
    # Show the window
    cv2.imshow("Convex Hull", frame1)

cv2.waitKey(0) & 0xFF
cv2.destroyAllWindows()

### 3.1.5
The above implementation only uses a minimum enclosing circle for labelling the detected image. There are more labelling options such as bounding boxes and convex hull which may come handy in various computer vision tasks.

**Your task**

- [ ] Implement a bounding rectangle and rotated rectangle on the detected contours by modifying the code in 3.1.4 
- [ ] Implement a convex hull on the detected contours by modifying the code in 3.1.4

**References**
- [Contour features in OpenCV](https://docs.opencv.org/4.x/dd/d49/tutorial_py_contour_features.html)
- [Creating Bounding boxes and circles for contours](https://docs.opencv.org/4.x/da/d0c/tutorial_bounding_rects_circles.html)
-[Convex Hull](https://docs.opencv.org/4.x/d7/d1d/tutorial_hull.html)

***
### 3.2 Object detection and Tracking with Video

### 3.2.1

Execute and observe the below code snippet which reads frames from a given video file at a pre-defined frame rate.

**Your task**
- [ ] Implement pre-processing and find contour of the moving blob on by modifiying the same code below
- [ ] Implement a suitable labelling mode (Enclosed Circle / bounding Box or convex hull) and display the resulted tracking on a seperated window

**Tip** : Use the same functions from 3.1. Avoid creating duplicate functions and unnecessarily lenghty code

In [4]:
# Libraries
from imutils.video import VideoStream
import time

# Lower and upper boundaries for HSV thresholds
videoLower = (95, 86, 6)        
videoUpper = (120, 255, 255)  

# Capture the video
vs = cv2.VideoCapture("test_video.avi")

# Script sleep
time.sleep(2.0)

# Variables
frame_rate = 30
prev = 0

while True :
    # Time elapsed
    time_elapsed = time.time() - prev

    # Per frame
    if time_elapsed > 1./frame_rate:
        # Read frames
        video_frame = vs.read()
        
        # Take the first frame
        video_frame = video_frame[1]
        
        # Check that there is frame data, otherwise the video has ended
        if video_frame is None:
            break
            
        # There is available frame, lets size it little bit
        video_frame = imutils.resize(video_frame, width=900)
        
        # Gaussian blur
        kernelSizeGaus = 11
        video_blurred = cv2.GaussianBlur(video_frame.copy(),(kernelSizeGaus,kernelSizeGaus),0)
        
        # Frame BGR -> HSV
        video_hsvFrame = cv2.cvtColor(video_blurred, cv2.COLOR_BGR2HSV)
        
        # Mask
        video_mask = cv2.inRange(video_hsvFrame,videoLower,videoUpper)
        
        # Erosion / Dialation -> Noice canselation which is caused from hand
        kernelSizeEroDia = 9
        eroDiaIterations = 1
        kernel = np.ones((kernelSizeEroDia, kernelSizeEroDia), np.uint8)
        video_maskEro = cv2.erode(video_mask, kernel, iterations=eroDiaIterations)
        video_mask = cv2.dilate(video_maskEro, kernel, iterations=eroDiaIterations)
        
        # Contour of the object 
        
        # Find contours / cnts is list
        video_cnts = find_cnts(video_mask)
        
        # Take the largest object array / numpy array
        video_c = max(video_cnts, key=cv2.contourArea)
        
        # Find the convex hull of a point set
        video_conhull = cv2.convexHull(video_c)
          
        # Draw points
        cv2.drawContours(video_frame,[video_conhull],0,(255,255,0),3)

        # Show frame
        cv2.imshow("Modified video", video_mask)
        cv2.imshow("Original video with contour", video_frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
        prev = time.time()
            
cv2.destroyAllWindows()   

---

###  3.3 Object detection and Tracking with ROS2

### 3.3.1 

In applications concerning robotic perception, so often visual data are obtained via camera sensors interfaced with ros. In this section you are provided with a ros-ignition simulation package which publish camera sensor data. The code snippet below subscribes to the camera sensor data and stream on to an OpenCV window. 

Assuming you have successfully build the package on your system using the instructions from handout, Open a New shell in you terminal and launch the simulation.

```
$ source /opt/ros/foxy/setup.bash
$ cd ~/ws
$ source install/setup.bash
$ ros2 launch walking_actor cam_world.launch.py
```

While the simulation is running in background , run the code snippet below and observe the result

- To close the cv2 window , select the window and press 'q'
- To stop the execution of code snippet, double press on 'i' in keyboard or interrupt the kernel from menu


**Your task**

- [ ] Modify the python code to track the walking actor. You can use any feature of the actor to track.
- [ ] Display the tracking in a seperate cv2 window


**References**
- [ROS2 python subscriber/publisher](https://docs.ros.org/en/foxy/Tutorials/Beginner-Client-Libraries/Writing-A-Simple-Py-Publisher-And-Subscriber.html)
- [rclpy API Documentation](https://docs.ros2.org/foxy/api/rclpy/api/execution_and_callbacks.html#module-rclpy.executors)


In [ ]:
# Import libraries
import sys
import rclpy
import imutils
from rclpy.node import Node
from cv_bridge import CvBridge
import numpy as np
import cv2
import time
from sensor_msgs.msg import Image, CameraInfo

# Bridge variable
bridge = CvBridge()

# Lower and upper boundaries for HSV thresholds
rosLower = (10, 60, 40)        
rosUpper = (140, 255, 255) 


class Get_Images(Node):

    # Constructor 
    def __init__(self):
        
        # Initialize the node
        super().__init__('Image_Subscriber')
        
        # Initialize the subscriber
        self.subscription_ = self.create_subscription(Image,'/camera', self.listener_callback,10)
        self.subscription_  # prevent unused variable warning
        
        # Timer function
        timer_period = 0.1  # seconds -> new image every 100ms 
        self.frame = None
        self.timer = self.create_timer(timer_period, self.timer_callback)
        self.K = True       # Ok variable

    # Subscribe callback function
    def listener_callback(self, msg):
        
        # Get the size of the frame and channel
        height = msg.height
        width = msg.width
        channel = msg.step//msg.width
        
        #frame = np.reshape(msg.data, (height, width, channel))
        
        # Set data to frame variable
        self.frame =  bridge.imgmsg_to_cv2(msg, desired_encoding='bgr8')
        
        # Log when image was received
        self.get_logger().info("Image Received")
        return self.frame
    
    def timer_callback(self):
        if self.K == True:

            if self.frame is None:
                return

            mod_frame = self.frame.copy()

            # Gaussian blur
            rosKernelSizeGaus = 15
            ros_blurred = cv2.GaussianBlur(mod_frame,(rosKernelSizeGaus,rosKernelSizeGaus),0)

            # Frame BGR -> HSV
            ros_hsvFrame = cv2.cvtColor(ros_blurred, cv2.COLOR_BGR2HSV)

            # Mask
            ros_mask = cv2.inRange(ros_hsvFrame,rosLower,rosUpper)

            # Erosion / Dialation -> Soft noice cancelation
            rosKernelSizeEroDia = 7
            rosEroDiaIterations = 1
            rosKernel = np.ones((rosKernelSizeEroDia, rosKernelSizeEroDia), np.uint8)
            rosMaskEro = cv2.erode(ros_mask, rosKernel, iterations=rosEroDiaIterations)
            ros_mask = cv2.dilate(rosMaskEro, rosKernel, iterations=rosEroDiaIterations)

            # Contour of the object 

            # Find contours / cnts is list
            ros_cnts = find_cnts(ros_mask)

            # Take the largest object array / numpy array
            ros_c = max(ros_cnts, key=cv2.contourArea)

            # Calculate xy position and add w and h to them (other corners)
            x,y,w,h = cv2.boundingRect(ros_c)

            # Draw the rectangle 
            cv2.rectangle(self.frame,(x,y),(x+w,y+h),(255,255,0),1)

            # Show frame
            cv2.imshow("Tracking", ros_mask)
            cv2.imshow("Original with contour",self.frame)
            key = cv2.waitKey(1) & 0xFF
            if key == ord("q"):
                self.get_logger().info('Closing stream window..')
                self.K = False
                cv2.destroyAllWindows()
   
    def stop_stream(self):
        self.get_logger().info('Stopping the stream ...')
        
### Find contours and display
def find_cnts(mask):
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    return cnts

try:
    
    rclpy.init(args=None)
    image_subscriber = Get_Images()
    rclpy.spin(image_subscriber)

except KeyboardInterrupt:
    # executes on keyboard kernal interrupt with double pressing button "i"
    image_subscriber.stop_stream()
    image_subscriber.destroy_node()
    rclpy.shutdown()
    